In [ ]:
import sys
sys.path.insert(0, "../")

# Downloading and processing data

There are two ways you can obtain OSM data in osm.pbf format:  
- Download it yourself: [parts of the world](https://download.geofabrik.de/), [cities](https://download.bbbike.org/osm/bbbike/), [adjustable area](https://extract.bbbike.org/) (via mail), [adjustable area](https://export.hotosm.org/en/v3/) (online), [planet](https://planet.maps.mail.ru/pbf/)
- Let the program download it for you

If the map is downloaded you can specify the filename:

In [ ]:
from offroad_routing.visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
filename = "../maps/kozlovo.osm.pbf"
bbox = [36.2, 56.5, 36.7, 56.7]
vgraph.compute_geometry(bbox=bbox, filename=filename)

Or, alternatively, you can only specify the bounding box, and the map will be downloaded automatically ([curl](https://curl.se/) & [osmosis](https://wiki.openstreetmap.org/wiki/Osmosis) required):

In [ ]:
bbox = [34, 59, 34.2, 59.1]
vgraph.compute_geometry(bbox=bbox)

Data inside this area can be processed using VisibilityGraph with chosen or default parameters.  
If not specified, optimal parameters will be computed by the algorithm.

In [ ]:
vgraph.prune_geometry(epsilon_polygon=0.003,
                      epsilon_linestring=0.001,
                      bbox_comp=10)

Computed data can also be saved in .h5 file to skip data processing the next time:

In [ ]:
vgraph.save_geometry("../maps/kozlovo.h5")

# Using precomputed data and building visibility graph

In [ ]:
from offroad_routing.visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
vgraph.load_geometry("../maps/user_area.h5")

Visibility graph can be built and (optionally) saved as networkx graph and (optionally) visualised using mplleaflet:

In [ ]:
%%time
import mplleaflet

G, fig = vgraph.build_graph(inside_percent=0,
                            multiprocessing=False,
                            graph=True,
                            map_plot=True)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

VisibilityGraph may also be used to find incident edges for a single point.  
This feature is used for pathfinding without graph building:

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

start = ((34.02, 59.01), None, None, None, None)
incidents = vgraph.incident_vertices(start)

fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)

# Building routes

In [4]:
from offroad_routing.visibility.visibility_graph import VisibilityGraph
from offroad_routing.pathfinding.astar import AStar

vgraph = VisibilityGraph()
vgraph.load_geometry("../maps/user_area.h5")

pathfinder = AStar(vgraph)
path = pathfinder.find((34.02, 59.01), (34.12, 59.09), default_weight=10, heuristic_multiplier=10)

Path can be viewed in coordinate format:

In [6]:
print(path.path())

[(34.02, 59.01), (34.0120063, 59.0077255), (34.0065139, 59.0027498), (34.0080357, 59.000195), (34.0308432, 59.0015589), (34.041719, 59.0002787), (34.0497156, 59.0003776), (34.0633361, 59.0067641), (34.0645012, 59.0097263), (34.0709203, 59.0111018), (34.0979576, 59.0191167), (34.1051888, 59.0182828), (34.1181669, 59.0234899), (34.1438569, 59.0481703), (34.1430844, 59.0706985), (34.1338791, 59.0775467), (34.1145924, 59.0822691), (34.1214683, 59.0864416), (34.12, 59.09)]


However, specialized tools can be used to save and visualize the path:

The following code saves the path to a gpx file and generates a link to view it online.

In [8]:
from offroad_routing.pathfinding.gpx_track import GpxTrack

track = GpxTrack(path)
track.write_file("track.gpx")
track.visualize()

Go to website: https://nakarte.me/#nktj=W3sibiI6ICIyMDIxLTA5LTE5IiwgInAiOiBbeyJuIjogIlN0YXJ0IiwgImx0IjogNTkuMDEsICJsbiI6IDM0LjAyfSwgeyJuIjogIkdvYWwiLCAibHQiOiA1OS4wOSwgImxuIjogMzQuMTJ9XSwgInQiOiBbW1s1OS4wMSwgMzQuMDJdLCBbNTkuMDA3NzI1NSwgMzQuMDEyMDA2M10sIFs1OS4wMDI3NDk4LCAzNC4wMDY1MTM5XSwgWzU5LjAwMDE5NSwgMzQuMDA4MDM1N10sIFs1OS4wMDE1NTg5LCAzNC4wMzA4NDMyXSwgWzU5LjAwMDI3ODcsIDM0LjA0MTcxOV0sIFs1OS4wMDAzNzc2LCAzNC4wNDk3MTU2XSwgWzU5LjAwNjc2NDEsIDM0LjA2MzMzNjFdLCBbNTkuMDA5NzI2MywgMzQuMDY0NTAxMl0sIFs1OS4wMTExMDE4LCAzNC4wNzA5MjAzXSwgWzU5LjAxOTExNjcsIDM0LjA5Nzk1NzZdLCBbNTkuMDE4MjgyOCwgMzQuMTA1MTg4OF0sIFs1OS4wMjM0ODk5LCAzNC4xMTgxNjY5XSwgWzU5LjA0ODE3MDMsIDM0LjE0Mzg1NjldLCBbNTkuMDcwNjk4NSwgMzQuMTQzMDg0NF0sIFs1OS4wNzc1NDY3LCAzNC4xMzM4NzkxXSwgWzU5LjA4MjI2OTEsIDM0LjExNDU5MjRdLCBbNTkuMDg2NDQxNiwgMzQuMTIxNDY4M10sIFs1OS4wOSwgMzQuMTJdXV19XQ==
